In [1]:
import os
import pandas
import numpy
import pickle
import pefile
import sklearn.ensemble as ek
from sklearn.model_selection import train_test_split
from sklearn import  tree, linear_model
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import svm
from sklearn.linear_model import LogisticRegression

In [2]:
dataset = pandas.read_csv("C:/Users/Acer/Documents/ML based Malicious Activity Detection/data.csv", low_memory=False)

In [3]:
dataset.head()

,Name,md5,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
0,memtest.exe,631ea355665f28d4707448e442fbf5b8,332,224,258,9,0,361984,115712,0,...,4,3.262823,2.568844,3.537939,8797.000000,216,18032,0,16,1
1,ose.exe,9d10f99a6712e28f8acd5641e3a7ea6b,332,224,3330,9,0,130560,19968,0,...,2,4.250461,3.420744,5.080177,837.000000,518,1156,72,18,1
2,setup.exe,4d92f518527353c0db88a70fddcfd390,332,224,3330,9,0,517120,621568,0,...,11,4.426324,2.846449,5.271813,31102.272727,104,270376,72,18,1
3,DW20.EXE,a41e524f8d45f0074fd07805ff0c9b12,332,224,258,9,0,585728,369152,0,...,10,4.364291,2.669314,6.400720,1457.000000,90,4264,72,18,1
4,dwtrig20.exe,c87e561258f2f8650cef999bf643a731,332,224,258,9,0,294912,247296,0,...,2,4.306100,3.421598,5.190603,1074.500000,849,1300,72,18,1


In [4]:
dataset.describe()

,Machine,SizeOfOptionalHeader,Characteristics,MajorLinkerVersion,MinorLinkerVersion,SizeOfCode,SizeOfInitializedData,SizeOfUninitializedData,AddressOfEntryPoint,BaseOfCode,...,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,LoadConfigurationSize,VersionInformationSize,legitimate
count,138047.000000,138047.000000,138047.000000,138047.000000,138047.000000,1.380470e+05,1.380470e+05,1.380470e+05,1.380470e+05,1.380470e+05,...,138047.000000,138047.000000,138047.000000,138047.000000,1.380470e+05,1.380470e+05,1.380470e+05,1.380470e+05,138047.000000,138047.000000
mean,4259.069274,225.845632,4444.145994,8.619774,3.819286,2.425956e+05,4.504867e+05,1.009525e+05,1.719561e+05,5.779845e+04,...,22.050700,4.000127,2.434541,5.521610,5.545093e+04,1.818082e+04,2.465903e+05,4.656750e+05,12.363115,0.299340
std,10880.347245,5.121399,8186.782524,4.088757,11.862675,5.754485e+06,2.101599e+07,1.635288e+07,3.430553e+06,5.527658e+06,...,136.494244,1.112981,0.815577,1.597403,7.799163e+06,6.502369e+06,2.124860e+07,2.608987e+07,6.798878,0.457971
min,332.000000,224.000000,2.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,332.000000,224.000000,258.000000,8.000000,0.000000,3.020800e+04,2.457600e+04,0.000000e+00,1.272100e+04,4.096000e+03,...,5.000000,3.458505,2.178748,4.828706,9.560000e+02,4.800000e+01,2.216000e+03,0.000000e+00,13.000000,0.000000
50%,332.000000,224.000000,258.000000,9.000000,0.000000,1.136640e+05,2.631680e+05,0.000000e+00,5.288300e+04,4.096000e+03,...,6.000000,3.729824,2.458492,5.317552,2.708154e+03,4.800000e+01,9.640000e+03,7.200000e+01,15.000000,0.000000
75%,332.000000,224.000000,8226.000000,10.000000,0.000000,1.203200e+05,3.850240e+05,0.000000e+00,6.157800e+04,4.096000e+03,...,13.000000,4.233051,2.696833,6.502239,6.558429e+03,1.320000e+02,2.378000e+04,7.200000e+01,16.000000,1.000000
max,34404.000000,352.000000,49551.000000,255.000000,255.000000,1.818587e+09,4.294966e+09,4.294941e+09,1.074484e+09,2.028711e+09,...,7694.000000,7.999723,7.999723,8.000000,2.415919e+09,2.415919e+09,4.294903e+09,4.294967e+09,26.000000,1.000000


In [5]:
dataset.groupby(dataset['legitimate']).size()


legitimate
0    96724
1    41323
dtype: int64

In [6]:
X = dataset.drop(['Name','md5','legitimate'],axis=1).values
y = dataset['legitimate'].values

In [7]:
extratrees = ek.ExtraTreesClassifier().fit(X,y)
model = SelectFromModel(extratrees, prefit=True)
X_new = model.transform(X)
nbfeatures = X_new.shape[1]

In [8]:
nbfeatures


14

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y ,test_size=0.2)


In [10]:
features = []
index = numpy.argsort(extratrees.feature_importances_)[::-1][:nbfeatures]

In [11]:
for f in range(nbfeatures):
    print("%d. Feature %s (%f)" % (f + 1, dataset.columns[2+index[f]], extratrees.feature_importances_[index[f]]))
    features.append(dataset.columns[2+f])

1. Feature DllCharacteristics (0.134170)
2. Feature Characteristics (0.108869)
3. Feature Machine (0.103164)
4. Feature ImageBase (0.068600)
5. Feature SectionsMaxEntropy (0.062892)
6. Feature Subsystem (0.053520)
7. Feature VersionInformationSize (0.052688)
8. Feature ResourcesMaxEntropy (0.045257)
9. Feature SizeOfOptionalHeader (0.044909)
10. Feature MajorSubsystemVersion (0.042546)
11. Feature ResourcesMinEntropy (0.040973)
12. Feature SizeOfStackReserve (0.025504)
13. Feature MajorOperatingSystemVersion (0.024757)
14. Feature SectionsMinEntropy (0.019786)


In [12]:
features

['Machine',
 'SizeOfOptionalHeader',
 'Characteristics',
 'MajorLinkerVersion',
 'MinorLinkerVersion',
 'SizeOfCode',
 'SizeOfInitializedData',
 'SizeOfUninitializedData',
 'AddressOfEntryPoint',
 'BaseOfCode',
 'BaseOfData',
 'ImageBase',
 'SectionAlignment',
 'FileAlignment']

In [13]:
model = { "DecisionTree":tree.DecisionTreeClassifier(max_depth=10),
         "RandomForest":ek.RandomForestClassifier(n_estimators=50),
         "ExtraTrees":ek.ExtraTreesClassifier(),
         "GNB":GaussianNB(),
         "LogisticRegression":LogisticRegression()   
}

In [14]:
results = {}
for algo in model:
    clf = model[algo]
    clf.fit(X_train,y_train)
    score = clf.score(X_test,y_test)
    print ("%s : %s " %(algo, score))
    results[algo] = score

DecisionTree : 0.9908366533864542 
RandomForest : 0.9944585295182905 
ExtraTrees : 0.99380659181456 
GNB : 0.6979355306048534 
LogisticRegression : 0.6978993118435349 


C:\Users\Acer\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [15]:
X_test

array([[3.32000000e+02, 2.24000000e+02, 2.58000000e+02, ...,
        2.62307817e+00, 5.91635626e+00, 0.00000000e+00],
       [3.32000000e+02, 2.24000000e+02, 2.59000000e+02, ...,
        2.71930208e+00, 5.29815574e+00, 0.00000000e+00],
       [3.32000000e+02, 2.24000000e+02, 8.45000000e+03, ...,
        3.54062702e+00, 3.54062702e+00, 1.80000000e+01],
       ...,
       [3.32000000e+02, 2.24000000e+02, 8.45000000e+03, ...,
        2.89787381e+00, 5.40199205e+00, 1.60000000e+01],
       [3.32000000e+02, 2.24000000e+02, 8.45000000e+03, ...,
        3.56688014e+00, 3.56688014e+00, 1.60000000e+01],
       [3.44040000e+04, 2.40000000e+02, 8.22600000e+03, ...,
        2.43495501e+00, 6.03104170e+00, 1.70000000e+01]])

In [16]:
winner = max(results, key=results.get)

In [17]:
joblib.dump(model[winner],"C:/Users/Acer/Documents/ML based Malicious Activity Detection/classifier/classifier.pkl")

['C:/Users/Acer/Documents/ML based Malicious Activity Detection/classifier/classifier.pkl']

In [18]:
open('C:/Users/Acer/Documents/ML based Malicious Activity Detection/classifier/features.pkl', 'wb').write(pickle.dumps(features))

267

In [19]:
clf = model[winner]
res = clf.predict(X_new)
mt = confusion_matrix(y, res)
print("False positive rate : %f %%" % ((mt[0][1] / float(sum(mt[0])))*100))
print('False negative rate : %f %%' % ( (mt[1][0] / float(sum(mt[1]))*100)))

False positive rate : 0.100285 %
False negative rate : 0.171817 %


In [20]:
# Load classifier
clf = joblib.load('C:/Users/Acer/Documents/ML based Malicious Activity Detection/classifier/classifier.pkl')
#load features
features = pickle.loads(open(os.path.join('C:/Users/Acer/Documents/ML based Malicious Activity Detection/classifier/features.pkl'),'rb').read())

In [21]:
%run "C:\Users\Acer\Documents\ML based Malicious Activity Detection\malware_test.py" "C:/Users/Acer/Documents/ML based Malicious Activity Detection/msedge.exe"

The file msedge.exe is legitimate


In [23]:
%run "C:\Users\Acer\Documents\ML based Malicious Activity Detection\malware_test.py" "C:/Users/Acer/Documents/ML based Malicious Activity Detection/Ikea-8.89.0.403.exe"

The file Ikea-8.89.0.403.exe is malicious
